In [1]:
refrence_path1 =r'C:\Users\trive\Desktop\pfizer\destination\bosnia\1. Label - Neurontin (gabapentin) capsules SPC_English.docx'
refrence_path2 =r'C:\Users\trive\Desktop\pfizer\destination\saudi\lable\check.docx'

In [2]:
import win32com.client
import pickle
def add_comment(word, doc, start_index,end_index, comment_text):
    doc.Activate()
    conflict_range = doc.Range(start_index,end_index)
    _ = doc.Comments.Add(conflict_range, comment_text)        
    return doc

def Medical_term(Medical_data,string):
    if string in Medical_data: 
        return True
    else:
        return False
def spell_grammar_errors(path):
    word = win32com.client.Dispatch("Word.Application")
    doc = word.Documents.Open(path)
    error_dict={}
    error_list=[]
    s_count=1
    g_count=1
    corrections = { 'spelling': [],'grammar':[]}
    with open('medical_terms.pkl', 'rb') as medict:
        Medical_data = pickle.load(medict)

    def rng_to_spell(rng):
        d = {'text': rng.Text, 'start': rng.Start, 'end': rng.End, 'suggestions': [],'type_of_language_error':'SPELL_ERROR'}
        for i in range(1, rng.GetSpellingSuggestions().Count+1):
            d['suggestions'].append(rng.GetSpellingSuggestions().Item(i).Name)
        return d   
    for rng in doc.Range().SpellingErrors:
        Check_med_word= Medical_term(Medical_data,rng.Text)
        if Check_med_word!=True:
            corrections['spelling'].append(rng_to_spell(rng))
            error_list.append(rng.Text)  
    def rng_to_grammar(rng):
        d = {'text': rng.Text, 'start': rng.Start, 'end': rng.End, 'suggestions': [],'type_of_language_error':'GRAMMAR_ERROR'}
        for i in range(1, rng.GetSpellingSuggestions().Count+1):
            d['suggestions'].append(rng.GetSpellingSuggestions().Item(i).Name)
        return d
    for rng in doc.Range().GrammaticalErrors:
        corrections['grammar'].append(rng_to_grammar(rng))    
    for correction in corrections['spelling']:
        if correction['type_of_language_error']=='SPELL_ERROR':
            comment = ""
            comment_dict={}
            if len(correction['suggestions']):
                comment = "Type of error: {}.\n Suggestions: {}".format(correction['type_of_language_error'], ", ".join(correction["suggestions"]))
                comment_dict["target_text"] = correction["text"]
                comment_dict["comment_text"] = comment
                comment_dict["conflict_type"] = "GRAMMAR_SPELLING"
                comment_dict["suggestions"] = correction["suggestions"]
                comment_dict["type_of_language_error"] = correction["type_of_language_error"]
            else:
                comment = "Type of error: {}.\nNo suggestions".format(correction["type_of_language_error"])
                comment_dict["target_text"] = correction["text"]
                comment_dict["comment_text"] = comment
                comment_dict["conflict_type"] = "GRAMMAR_SPELLING"
                comment_dict["suggestions"] = []
                comment_dict["type_of_language_error"] = correction["type_of_language_error"]
            if s_count==1:
                add_comment(word, doc, correction["start"], correction["end"],comment)
            else:
                add_comment(word, doc, correction["start"]+s_count-1, correction["end"]+s_count-1,comment)
        s_count=s_count+1
        
    for correction in corrections['grammar']:       
        if correction['type_of_language_error']=='GRAMMAR_ERROR':
            comment_dict = {}
            comment = ""
            if len(correction['suggestions']):
                comment = "Type of error: {}.\nSuggestions: {}".format(correction['type_of_language_error'], ", ".join(correction["suggestions"]))
                comment_dict["target_text"] = correction["text"]
                comment_dict["comment_text"] = comment
                comment_dict["conflict_type"] = "GRAMMAR_SPELLING"
                comment_dict["suggestions"] = correction["suggestions"]
                comment_dict["type_of_language_error"] = correction["type_of_language_error"]
            else:
                comment = "Type of error: {}.\nNo suggestions".format(correction["type_of_language_error"])
                comment_dict["target_text"] = correction["text"]
                comment_dict["comment_text"] = comment
                comment_dict["conflict_type"] = "GRAMMAR_SPELLING"
                comment_dict["suggestions"] = []
                comment_dict["type_of_language_error"] = correction["type_of_language_error"]
            error_list.append(comment_dict)
            if g_count==1:
                add_comment(word, doc, correction["start"], correction["end"],comment)
            else:
                add_comment(word, doc, correction["start"]+g_count-1, correction["end"]+g_count-1,comment)
        g_count=g_count+1          
    return corrections         

In [4]:
%%time
spell_grammar_errors(refrence_path1)

Wall time: 12min 54s


{'spelling': [{'text': 'Whitehard',
   'start': 825,
   'end': 834,
   'suggestions': ['White hard'],
   'type_of_language_error': 'SPELL_ERROR'},
  {'text': 'titratingthe',
   'start': 2851,
   'end': 2863,
   'suggestions': ['titrating the'],
   'type_of_language_error': 'SPELL_ERROR'},
  {'text': 'ofgabapentin',
   'start': 3147,
   'end': 3159,
   'suggestions': [],
   'type_of_language_error': 'SPELL_ERROR'},
  {'text': 'clinicalstudies',
   'start': 3437,
   'end': 3452,
   'suggestions': ['clinical studies'],
   'type_of_language_error': 'SPELL_ERROR'},
  {'text': 'upwardtitration',
   'start': 3749,
   'end': 3764,
   'suggestions': ['upward titration'],
   'type_of_language_error': 'SPELL_ERROR'},
  {'text': 'termclinical',
   'start': 3960,
   'end': 3972,
   'suggestions': ['term clinical'],
   'type_of_language_error': 'SPELL_ERROR'},
  {'text': 'timeinterval',
   'start': 4054,
   'end': 4066,
   'suggestions': ['time interval'],
   'type_of_language_error': 'SPELL_ERROR'}